In [1]:
'''优先队例求出一个无序整数序列中的第k小的元素'''
import heapq
import random


def find_kth_smallest(arr, k):
    if k <= 0 or k > len(arr):
        return None
    heap = []
    for num in arr:
        if len(heap) < k:  # 如果最大堆的长度小于k，直接添加
            heapq.heappush(heap, -num)  # heapq默认最小堆，取负数变成最大堆
        else:
            if num < -heap[0]:  # 如果新来的元素比堆顶元素更小
                heapq.heappop(heap)  # 踢出原来的堆顶
                heapq.heappush(heap, -num)  # 加入新元素，heapq自动调整大小，堆顶永远最大
    return -heap[0]


seed = 202211672411
random.seed(seed)

n = 10
arr = [random.randint(1, 100) for _ in range(n)]
print("生成的数组:", arr)

for k in range(1, 6):
    sorted_arr = sorted(arr)
    expected = sorted_arr[k - 1] if k <= len(arr) else None
    result = find_kth_smallest(arr, k)
    print(f"\n排序后的数组: {sorted_arr}")
    print(f"第{k}小的元素: {result}")

生成的数组: [87, 58, 35, 86, 85, 89, 89, 52, 90, 20]

排序后的数组: [20, 35, 52, 58, 85, 86, 87, 89, 89, 90]
第1小的元素: 20

排序后的数组: [20, 35, 52, 58, 85, 86, 87, 89, 89, 90]
第2小的元素: 35

排序后的数组: [20, 35, 52, 58, 85, 86, 87, 89, 89, 90]
第3小的元素: 52

排序后的数组: [20, 35, 52, 58, 85, 86, 87, 89, 89, 90]
第4小的元素: 58

排序后的数组: [20, 35, 52, 58, 85, 86, 87, 89, 89, 90]
第5小的元素: 85


In [2]:
'''动态数组+哈希'''


class EfficientDataStructure:
    def __init__(self):
        self.array = []  # 动态数组存储元素
        # 动态数组在初始化时不需要指定固定大小，而是在插入元素时自动扩展容量。Python中的列表就是一种典型的动态数组
        self.value_to_index = {}  # 哈希表：值 → 数组索引

    def insert(self, value):
        """插入元素"""
        if value in self.value_to_index:  # 如果要插入的元素在字典里面了，为了不重复，不插入
            return False
        self.array.append(value)  # 没有重复就插入动态数组
        self.value_to_index[value] = len(self.array) - 1  # 字典的键是元素，值是在动态数组中的索引
        return True

    def delete(self, value):
        """删除元素"""
        if value not in self.value_to_index:  # 如果字典的键中不存在这个元素，即动态数组中没有这个元素
            return False  # 那就删除不了
        index = self.value_to_index[value]  # 否则先从字典中找个要删除元素的索引
        last_val = self.array[-1]  # 获取数组末尾元素

        # 交换待删除元素与末尾元素
        self.array[index], self.array[-1] = self.array[-1], self.array[index]
        self.value_to_index[last_val] = index

        # 交换后删除的时间复杂度是O(1)，不然后面的元素都要向前面移动
        self.array.pop()
        del self.value_to_index[value]  # 因为待删除的元素已经在最后，且pop出去了，所以不需要更新字典
        return True

    def get_by_value(self, value):
        """按值查找是否存在"""
        return value in self.value_to_index

    def get_by_index(self, index):
        """按序号查找元素"""
        if index < 0 or index >= len(self.array):
            return None
        return self.array[index]  # 直接通过字典进行查找

    def size(self):
        return len(self.array)


def generate_test_data(data_type='int', size=1000, seed=202211672411):
    """生成测试数据集"""
    random.seed(seed)
    data = set()

    if data_type == 'int':
        # 生成唯一随机整数
        while len(data) < size - 1:
            data.add(random.randint(0, 10 ** 12))
        # 添加学号
        student_id = 202211672411
        data.add(student_id)
    else:
        while len(data) < size - 1:
            s = ''.join(random.choices('abcdefghijklmnopqrstuvwxyz', k=10))
            data.add(s)
        # 添加姓名拼音
        data.add("LiChuanTao")

    return list(data)


# 测试示例
if __name__ == "__main__":
    test_data = generate_test_data(data_type='int', size=1000)  # 生成1000个各不相同的整数

    # 初始化数据结构
    ds = EfficientDataStructure()
    for num in test_data:
        ds.insert(num)

    # 按值查找
    student_id = 202211672411
    print(f"黎川滔的学号 {student_id} 是否在数组中？", ds.get_by_value(student_id))

    # 按序号查找
    print("数组中索引为 0 的元素：", ds.get_by_index(0))

    # 删除元素
    ds.delete(student_id)
    print("删除黎川滔学号后数组的大小", ds.size())
    print(f"黎川滔的学号 {student_id} 在删除后是否还存在？", ds.get_by_value(student_id))

黎川滔的学号 202211672411 是否在数组中？ True
数组中索引为 0 的元素： 222225760264
删除黎川滔学号后数组的大小 999
黎川滔的学号 202211672411 在删除后是否还存在？ False


In [4]:
'''线性探测'''


class HashTable:
    def __init__(self, size):
        self.size = size  # 哈希表的大小
        self.table = [None] * size  # 初始化哈希表，所有位置为空

    def hash_function(self, key):
        """哈希函数：计算键的哈希值"""
        return key % self.size

    def insert(self, key, value):
        """插入键值对到哈希表中"""
        index = self.hash_function(key)  # 计算初始索引
        while self.table[index] is not None:  # 如果当前位置被占用
            if self.table[index][0] == key:  # 如果键已存在，更新值
                self.table[index] = (key, value)
                return
            index = (index + 1) % self.size  # 线性探测：检查下一个位置
        self.table[index] = (key, value)  # 找到空闲位置，插入键值对

    def search(self, key):
        """查找键对应的值"""
        index = self.hash_function(key)  # 计算初始索引
        while self.table[index] is not None:  # 如果当前位置被占用
            if self.table[index][0] == key:  # 如果找到键
                return self.table[index][1]  # 返回对应的值
            index = (index + 1) % self.size  # 线性探测：检查下一个位置
        return None  # 未找到键

    def delete(self, key):
        """删除键值对"""
        index = self.hash_function(key)  # 计算初始索引
        while self.table[index] is not None:  # 如果当前位置被占用
            if self.table[index][0] == key:  # 如果找到键
                self.table[index] = None  # 删除键值对
                self.rehash(index)  # 重新哈希后续的键值对
                return
            index = (index + 1) % self.size  # 线性探测：检查下一个位置

    def rehash(self, start_index):
        """重新哈希从 start_index 开始的所有键值对"""
        index = (start_index + 1) % self.size  # 从下一个位置开始
        while self.table[index] is not None:  # 如果当前位置被占用
            key, value = self.table[index]  # 获取键值对
            self.table[index] = None  # 清空当前位置
            self.insert(key, value)  # 重新插入键值对
            index = (index + 1) % self.size  # 检查下一个位置

    def __str__(self):
        """打印哈希表内容"""
        return "\n".join(f"Index {i}: {self.table[i]}" for i in range(self.size))

if __name__ == "__main__":
    ht = HashTable(10)  # 创建大小为 10 的哈希表

    # 插入键值对
    ht.insert(10, "A")
    ht.insert(20, "B")
    ht.insert(30, "C")
    ht.insert(15, "D")  # 哈希冲突，使用线性探测
    print("哈希表内容：")
    print(ht)

    # 查找键
    print("\n查找键 20 的值：", ht.search(20))
    print("查找键 15 的值：", ht.search(15))

    # 删除键
    ht.delete(20)
    print("\n删除键 20 后的哈希表内容：")
    print(ht)

哈希表内容：
Index 0: (10, 'A')
Index 1: (20, 'B')
Index 2: (30, 'C')
Index 3: None
Index 4: None
Index 5: (15, 'D')
Index 6: None
Index 7: None
Index 8: None
Index 9: None

查找键 20 的值： B
查找键 15 的值： D

删除键 20 后的哈希表内容：
Index 0: (10, 'A')
Index 1: (30, 'C')
Index 2: None
Index 3: None
Index 4: None
Index 5: (15, 'D')
Index 6: None
Index 7: None
Index 8: None
Index 9: None


In [2]:
'''基数排序'''
import random


def generate_student_ids(num_students=50000):
    """生成指定数量的学生学号"""
    student_ids = []
    for _ in range(num_students):
        year = random.randint(2020, 2023)  # 入学年份：2020-2023
        dept = random.randint(1000, 9999)  # 学院与专业：1000-9999
        class_num = random.randint(1, 3)  # 班级序号：1-3
        student_num = random.randint(1, 99)  # 班级内部学号：1-99
        # 格式化学号：入学年份（4位） + 学院与专业（4位） + 班级序号（2位） + 班级内部学号（2位）
        student_id = f"{year}{dept:04d}{class_num:02d}{student_num:02d}"
        student_ids.append(student_id)
    return student_ids


def radix_sort(arr):
    """基数排序"""
    max_length = len(max(arr, key=len))  # 获取最长学号的长度
    for i in range(max_length - 1, -1, -1):  # 从最低位到最高位依次排序
        buckets = [[] for _ in range(10)]  # 初始化10个桶（0-9）
        for num in arr:
            # 获取当前位的数字，如果不足则补0
            digit = int(num[i]) if i < len(num) else 0
            buckets[digit].append(num)  # 将数字放入对应的桶中
        arr = [num for bucket in buckets for num in bucket]  # 将桶中的数字按顺序合并
    return arr


# 测试代码
if __name__ == "__main__":
    # 生成50000个学生学号
    student_ids = generate_student_ids(50000)
    print("前10个未排序的学号：", student_ids[:10])

    # 基数排序
    sorted_student_ids = radix_sort(student_ids)
    print("\n前10个排序后的学号：", sorted_student_ids[:10])

    # 检查是否有序
    is_sorted = all(sorted_student_ids[i] <= sorted_student_ids[i + 1] for i in range(len(sorted_student_ids) - 1))
    print("\n学号是否有序：", is_sorted)

前10个未排序的学号： ['202265780118', '202344260133', '202096380286', '202372880330', '202062710219', '202228530276', '202029580158', '202274100304', '202398020376', '202394850124']

前10个排序后的学号： ['202010000395', '202010010149', '202010010178', '202010010294', '202010020125', '202010020271', '202010030282', '202010040178', '202010040254', '202010040277']

学号是否有序： True


In [1]:
nums = [3, 2, 2, 3]
val = 3
nums = [x for x in nums if x != val]
print(len(nums))
print(nums)

2
[2, 2]


25
